In [5]:
import numpy as np

In [47]:
class RCN_binomial:
    def __init__(self, interest_rate, period_length, initial_price, dividend_yield, up_factor, down_factor,
                 payment_dates, annualized_coupon, exercise_price, Simple=True, Callable=False, barrier_level=None):
        # binomial model parameters
        self.r = interest_rate
        self.Delta = period_length
        self.i0 = initial_price
        self.delta = dividend_yield
        self.U = up_factor
        self.D = down_factor
        
        self.gamma = np.exp(-self.r*self.Delta)
        self.q = (1/self.gamma - self.D)/(self.U - self.D)
        
        # RCN characteristics
        self.simple = Simple
        self.callable = Callable
        self.T = payment_dates[len(payment_dates)-1]
        self.c = annualized_coupon
        self.alpha = exercise_price
        self.beta = barrier_level
    
    def terminal_payoffs(self):
        # with Markov property
        t = np.arange(1, self.T+1, 1)
        P = np.zeros((self.T+1, self.T+1))
        P[0,0] = self.i0
        div = 1
        if self.delta == 0:
            div = 0
        for j in t:
            for i in range(j+1):
                if i == 0:
                    P[i,j] = P[i,j-1] * (self.U - div*np.exp(-self.delta*self.Delta))
                else:
                    P[i,j] = P[i-1,j-1] * (self.D - div*np.exp(-self.delta*self.Delta))
        H = np.maximum(P[:,self.T] - self.alpha*self.i0, 0)
        return H
    
    def price_option(self, put=True):
        if self.q <= 0 or self.q >= 1:
            return
        else:
            # with Markov propoerty
            t = np.arange(self.T-1,-1,-1)
            P = np.zeros((self.T+1, self.T+1))
            P[:,self.T] = self.terminal_payoffs()
            for j in t:
                for i in range(j+1):
                    P[i,j] = self.gamma*( self.q*P[i, j+1] + (1-self.q)*P[i+1, j+1] )
            return P[0,0]

In [48]:
rcn_b = RCN_binomial(interest_rate=0, period_length=1, initial_price=10, dividend_yield=0, up_factor=2, down_factor=1/2,
                 payment_dates=[1,2,3], annualized_coupon=0, exercise_price=1, Simple=True, Callable=False, barrier_level=None)

In [49]:
rcn_b.terminal_payoffs()

array([70., 10.,  0.,  0.])

In [50]:
rcn_b.price_option()

4.814814814814814